# Experiment Notebook for CV2

This notebook is a development workspace to add or subtract features during model development. Once a set of changes is ready to be experimented with, it should be cleaned, copied and saved to a new notebook that can be run end to end with no errors and committed in a separate git commit. For example, "fastai2_unet_trainer_cv2-1channel-baseline.ipynb" is an experiment that should not be changed in version control once committed but cells can be edited to inspect the results on your local machine.

In [ ]:
from torchvision.models import resnet18, resnet34, resnet50
bs_d ={512:4, 256:32, 224:32, 128:64, 64:256}
lr_d = {512:3e-4, 256:1e-3, 224:3e-3, 128:3e-3, 64:1e-2}
arch_d = {18: resnet18, 34: resnet34, 50: resnet50}


size=224
bs = bs_d[size]
n="all"
arch=34
epochs = 30
freeze = 0
negative_sample_count = 0
negative_sample_count_val = 0
area_thresh = 0
classes_to_remove=[
    "ambiguous",
    ]
classes_to_remap ={
    # "old_vessel": "recent_vessel",
    # "coincident_vessel": "recent_vessel",
}

In [ ]:
from ceruleanml import data
from ceruleanml import evaluation
from ceruleanml import preprocess
from fastai.data.block import DataBlock
from fastai.vision.data import ImageBlock, MaskBlock
from fastai.vision.augment import aug_transforms, Resize
from fastai.vision.learner import unet_learner
from fastai.data.transforms import IndexSplitter
from fastai.metrics import DiceMulti, Dice, accuracy_multi, PrecisionMulti, RecallMulti
from ceruleanml.coco_load_fastai import record_collection_to_record_ids, get_image_path, record_to_mask
from fastai.callback.fp16 import MixedPrecision
from fastai.callback.tensorboard import TensorBoardCallback
from fastai.vision.core import PILImageBW
from datetime import datetime
from pathlib import Path
import os, random
from icevision.visualize import show_data
import torch
from fastai.callback.tracker import EarlyStoppingCallback, SaveModelCallback
import skimage.io as skio
import numpy as np
from math import log


In [ ]:
def get_held_scenes(f_paths):
    held_scenes = []
    for f_path in f_paths:
        with open(f_path) as f:
            data = f.read().split("\n")
            held_scenes += [line.split("/")[-1] for line in data]
    return held_scenes
test_scenes = f"/root/data/partitions/test_scenes.txt"
val_scenes = f"/root/data/partitions/val_scenes.txt"
held_scenes = get_held_scenes([test_scenes, val_scenes])

In [ ]:

if not 'this_cell_has_been_run_already' in locals():

    ### Parsing COCO Dataset with Icevision

    mount_path = "/root/"
    train_set = "train-with-context-512"
    tiled_images_folder_train = "tiled_images"
    json_name_train = "instances_TiledCeruleanDatasetV2.json"

    coco_json_path_train = f"{mount_path}/partitions/{train_set}/{json_name_train}"
    tiled_images_folder_train = f"{mount_path}/partitions/{train_set}/{tiled_images_folder_train}"
    val_set = "val-with-context-512"
    tiled_images_folder_val= "tiled_images"
    json_name_val = "instances_TiledCeruleanDatasetV2.json"
    coco_json_path_val= f"{mount_path}/partitions/{val_set}/{json_name_val}"
    tiled_images_folder_val = f"{mount_path}/partitions/{val_set}/{tiled_images_folder_val}"

    ## looking at area distribution to find area threshold

    # df = preprocess.get_area_df(coco_json_path_train, tiled_images_folder_train)
    # df
    
    record_collection_with_negative_small_filtered_train = preprocess.load_set_record_collection(
    coco_json_path_train, tiled_images_folder_train, area_thresh, negative_sample_count_val, preprocess=False, 
    classes_to_remap=classes_to_remap, classes_to_remove=classes_to_remove, held_scenes=held_scenes)
    
    record_collection_with_negative_small_filtered_val = preprocess.load_set_record_collection(
    coco_json_path_val, tiled_images_folder_val, area_thresh, negative_sample_count_val, preprocess=True,
    classes_to_remap=classes_to_remap, classes_to_remove=classes_to_remove)

    record_ids_train = record_collection_to_record_ids(record_collection_with_negative_small_filtered_train)
    record_ids_val = record_collection_to_record_ids(record_collection_with_negative_small_filtered_val)

    assert len(set(record_ids_train)) + len(set(record_ids_val)) == len(record_ids_train) + len(record_ids_val)

    train_val_record_ids = record_ids_train + record_ids_val
    combined_record_collection = record_collection_with_negative_small_filtered_train + record_collection_with_negative_small_filtered_val

    def get_val_indices(combined_ids, val_ids):
        return list(range(len(combined_ids)))[-len(val_ids):]

    #show_data.show_records(random.choices(combined_train_records, k=9), ncols=3)

    ### Constructing a FastAI DataBlock that uses parsed COCO Dataset from icevision parser. aug_transforms can only be used with_context=True

    val_indices = get_val_indices(train_val_record_ids, record_ids_val)

    def get_image_by_record_id(record_id):
        return get_image_path(combined_record_collection, record_id)

    def get_mask_by_record_id(record_id):
        return record_to_mask(combined_record_collection, record_id)

    this_cell_has_been_run_already = True
else:
    print('skipped')


In [ ]:
batch_transfms = [*aug_transforms(flip_vert=True, max_rotate=180, max_warp=0.1, size=size)]
coco_seg_dblock = DataBlock(
        blocks=(ImageBlock, MaskBlock(codes=data.class_list)), # ImageBlock is RGB by default, uses PIL
        get_x=get_image_by_record_id,
        splitter=IndexSplitter(val_indices),
        get_y=get_mask_by_record_id,
        batch_tfms=batch_transfms,
        item_tfms = Resize(size),
        n_inp=1
    )


dls = coco_seg_dblock.dataloaders(source=train_val_record_ids, batch_size=bs)

# dls.show_batch()

### Fastai2 Trainer

In [ ]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d_%b_%Y_%H_%M_%S")
experiment_dir =  Path(f'{mount_path}/experiments/cv2/'+timestampStr+'_fastai_unet/')
experiment_dir.mkdir(exist_ok=True)
print(experiment_dir)

In [ ]:
cbs = [TensorBoardCallback(projector=False, trace_model=False), 
       # SaveModelCallback(monitor="valid_loss", with_opt=True),
       # EarlyStoppingCallback(monitor='valid_loss', min_delta=0.005, patience=10) 
       ]

learner = unet_learner(dls, arch_d[arch], metrics=[DiceMulti, Dice],
                       model_dir=experiment_dir,
                       cbs=cbs) #cbs=cbs# SaveModelCallback saves model when there is improvement
# lr = learner.lr_find()
running_total_epochs = {}

In [ ]:
lr = lr_d[size]
# lr = learner.lr_find()
# lr

In [ ]:
print("starting from running total", running_total_epochs)
print("size", size)
print("batch size", bs)
print("arch", arch)
print("lr", lr)
print("n chips", n)
print("epochs", epochs)
print("freeze", freeze)

learner.fine_tune(epochs, lr, freeze_epochs=freeze) # cbs=cbs
learner.show_results()

running_total_epochs[size] = sum(filter(None,[running_total_epochs.get(size),epochs,freeze]))

# Progressive Resizing

In [ ]:
for size in [224]*20:
    bs = bs_d[size]
    lr = lr_d[size]

    batch_transfms = [*aug_transforms(flip_vert=True, max_rotate=180, max_warp=0.1, size=size)]
    coco_seg_dblock = DataBlock(
            blocks=(ImageBlock, MaskBlock(codes=data.class_list)), # ImageBlock is RGB by default, uses PIL
            get_x=get_image_by_record_id,
            splitter=IndexSplitter(val_indices),
            get_y=get_mask_by_record_id,
            batch_tfms=batch_transfms,
            item_tfms = Resize(size),
            n_inp=1
        )
    learner.dls = coco_seg_dblock.dataloaders(source=train_val_record_ids, batch_size=bs)
    print("starting from running total", running_total_epochs)
    print("image size", size)
    print("batch size", bs)
    print("arch", arch)
    print("lr", lr)
    print("n chips", n)
    print("epochs", epochs)
    print("freeze", freeze)

    learner.fine_tune(epochs, lr, freeze_epochs=freeze) # cbs=cbs
    learner.show_results()

    running_total_epochs[size] = sum(filter(None,[running_total_epochs.get(size),epochs,freeze]))
    checkpoint = learner

# Model Inference and Result Evaluation

In [ ]:
with learner.no_bar():
    val_arrs = []
    class_preds = []
    for batch_tuple in dls.train:
        for img, val_mask in zip(batch_tuple[0], batch_tuple[1]):
            semantic_mask = val_mask.cpu().detach().numpy()
            class_pred = learner.predict(img.cpu())
            class_pred = class_pred[0].cpu().detach().numpy()
            val_arrs.append(semantic_mask)
            class_preds.append(class_pred)
            break

# inspecting preds

In [ ]:
records_of_interest = []
for record in record_collection_with_negative_small_filtered_train:
    if "S1A_IW_GRDH_1SDV_20200724T020738_20200724T020804_033590_03E494_B457" in str(record.common.filepath):
        records_of_interest.append(record)

In [ ]:
arr = skio.imread(records_of_interest[0].common.filepath)

In [ ]:
arr = np.moveaxis(arr, 2, 0)

In [ ]:
b = next(iter(dls.train))
val_mask  = b[1][0]
semantic_mask = val_mask.cpu().detach().numpy()
img  = b[0][0]

In [ ]:
import skimage.io as skio
skio.imshow(semantic_mask)

In [ ]:
skio.imshow(np.moveaxis(img.cpu().detach().numpy(), 0, 2)[:,:,0])

In [ ]:
_,_,preds=learner.get_preds(dl=[b], with_decoded=True)
dls.show_results(b, preds, max_n=1)

In [ ]:
outputs = learner.predict(img.cpu())

In [ ]:
valid_loss, dice_multi, dice = learner.validate()

In [ ]:
from ceruleanml.inference import save_fastai_model_state_dict_and_tracing, load_tracing_model, test_tracing_model_one_batch, logits_to_classes
save_template = f'test_{bs}_{arch}_{size}_{round(dice_multi,3)}_{epochs}.pt'
state_dict_pth, tracing_model_gpu_pth, tracing_model_cpu_pth  = save_fastai_model_state_dict_and_tracing(learner, dls, save_template, experiment_dir)

In [ ]:
experiment_dir

In [ ]:
!cp -r {experiment_dir} /root/data/experiments/cv2/

After loading numpy array batch with the shape above, import Torch and call torch.Tensor on the numpy array batch of tiles

In [ ]:
model = torch.jit.load(tracing_model_cpu_pth)
#these are equivalent
# inference on a tensor
# b_result = model(b[0].cpu())

# inference on numpy array converted to tensor
b_result = model(torch.Tensor(b[0].cpu().detach().numpy())) 

In [ ]:
skio.imshow(b_result[0].sigmoid().detach().numpy()[0])

In [ ]:
b_result.shape

In [ ]:
from ceruleanml.inference import logits_to_classes

In [ ]:
conf, classes = logits_to_classes(tile)

In [ ]:
def logits_to_classes(out_batch_logits):
    """returns the confidence scores of the max confident classes
    and an array of max confident class ids.
    """
    probs = torch.nn.functional.softmax(out_batch_logits, dim=1)
    conf, classes = torch.max(probs, 1)
    return (conf, classes)

In [ ]:
tile.shape

In [ ]:
classes.shape

In [ ]:
type(conf)

In [ ]:
#argmax to get category index where confidence is highest
# confidence is returned after applying sigmoid to the logits
# we only apply sigmoid on an individual tile from the batch!
# tile = b_result[0,:,:,:]
# indices = np.argmax(tile.softmax().cpu().detach().numpy(), axis=0)

In [ ]:
conf_thresh=.3

In [ ]:
# cpu_tile_confs= tile.sigmoid().cpu().detach().numpy()

In [ ]:
def apply_conf_threshold(conf, classes, conf_threshold):
    high_conf_mask = torch.any(torch.where(conf> conf_thresh, 1, 0), axis=0)
    return torch.where(high_conf_mask, classes, 0)
    

In [ ]:
apply_conf_threshold(conf, classes, conf_thresh)

In [ ]:
high_conf_mask = np.any(np.where(conf> conf_thresh, 1, 0), axis=0)

In [ ]:
skio.imshow(classes.cpu().detach().numpy())

In [ ]:
skio.imshow(tile.cpu().detach().numpy()[0])

In [ ]:
skio.imshow(tile.cpu().detach().numpy()[1])

In [ ]:
data.class_idx_dict

In [ ]:
skio.imshow(tile.cpu().detach().numpy()[6])

In [ ]:
skio.imshow(conf.cpu().detach().numpy())

In [ ]:
skio.imshow(np.where(high_conf_mask, classes, 0))

# CM is bugged because predict is bugged

In [ ]:
evaluation.get_cm_for_learner(dls, learner, mount_path)

In [ ]:
validation = learner.validate()

# We save the best model in a variety of formats for loading later. Eval on Torchscript model still being debugged

In [ ]:
save_template = f'test_{bs}_{arch}_{size}_{round(validation[1],3)}_{epochs}.pt'

In [ ]:
from ceruleanml.inference import save_fastai_model_state_dict_and_tracing, load_tracing_model, test_tracing_model_one_batch, logits_to_classes

state_dict_pth, tracing_model_gpu_pth, tracing_model_cpu_pth  = save_fastai_model_state_dict_and_tracing(learner, dls, save_template, experiment_dir)

model = torch.load(tracing_model_cpu_pth)

In [ ]:
def get_cm_for_torchscript_model(dls, model, save_path):
"""
the torchscript model when it is loaded operates on batches, not individual images
this doesn't support eval on negative samples if they are in the dls, 
since val masks don't exist with neg samples. need to be constructed with np.zeros

returns cm and f1 score
"""
val_arrs = []
class_preds = []
for batch_tuple in dls.valid:
    semantic_masks_batch = batch_tuple[1].cpu().detach().numpy()
    class_pred_batch = model(batch_tuple[0].cpu())
    class_pred_batch = class_pred_batch.cpu().detach().numpy()
    val_arrs.extend(semantic_masks_batch)
    class_preds.append(class_pred_batch)
return evaluation.cm_f1(val_arrs, class_preds, save_path) # todo add normalize false

In [ ]:
get_cm_for_torchscript_model(dls, model, mount_path)

In [ ]:
result = learner.get_preds(dl=dls[0])

In [ ]:
learner.get_preds??

In [ ]:
len(pred_arrs)

In [ ]:
target_label,prediction_arr, activations = pred_arrs[0]

In [ ]:
skio.imshow(target_label.cpu().detach().numpy())

In [ ]:
skio.imshow(base_img.cpu().detach().numpy()[0])

In [ ]:
skio.imshow(base_img.cpu().detach().numpy()[1])

In [ ]:
skio.imshow(base_img.cpu().detach().numpy()[1])

In [ ]:
skio.imshow(base_img.cpu().detach().numpy())

In [ ]:
array([      60.73,       190.3,      4.3598]) # means
array([     16.099,      17.846,       9.603]) # stats